In [5]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
%matplotlib inline

# the following names should be changed for each use
# specify the location and name of the DEM
fdir = "D:\\crs\\proj\\2017_Ontario\\2017-07-13_Crescent_Charles\\"
fname = ("2017-07-13_Crescent_Charles_DEM_v3_5cm.tif")
# specify the name of the transect file (assumed to be in same directory)
tfname = '2017-07-12-13_Crescent_Bar_Lake_Bluffs_trans.txt'
# text for plot titles
ttext = "Crescent Bar / Charles Point"

In [6]:
# read DEM channel 1
with rasterio.open(fdir+fname) as dem:
    z = dem.read(1)
    
print(np.shape(z))

(13360, 38480)


In [7]:
# coordinate reference system
print(dem.crs)
# data transform
print(dem.transform)
# upper left corner
print('upper left: ',dem.transform*(0,0))
# lower right corner
print('lower right: ',dem.transform*(dem.width, dem.height))

# find the array index of a point by inverting the transform (!)
# note order: (easting, northing)
# print( ~dem.transform*(341294.0, 4793569.0) )

CRS({'init': 'epsg:26918'})
| 0.05, 0.00, 340054.00|
| 0.00,-0.05, 4793059.00|
| 0.00, 0.00, 1.00|
upper left:  (340054.0, 4793059.0)
lower right:  (341978.0, 4792391.0)


In [8]:
# read in the transect points
tfname = '2017-07-12-13_Crescent_Bar_Lake_Bluffs_trans.txt'
column_names = ['id','n','e','elev','lat','lon','label']
import pandas as pd
df=pd.read_csv(fdir+tfname,header = None, names = column_names)
print(df)
print("Read: ",len(df.index),' lines.')

      id             n            e      elev        lat        lon     label
0     11  4.793302e+06  342642.8825  101.9704  43.276098 -76.939247    BLUFF1
1     12  4.793294e+06  342644.3948  102.1922  43.276029 -76.939226    BLUFF2
2     13  4.793292e+06  342631.5170  104.3634  43.276012 -76.939384    BLUFF3
3     14  4.793293e+06  342620.3511  104.9136  43.276020 -76.939522    BLUFF5
4     15  4.793277e+06  342628.2735  104.6322  43.275870 -76.939420    BLUFF6
5     16  4.793272e+06  342609.1304  105.3865  43.275822 -76.939654    BLUFF7
6     17  4.793255e+06  342608.0334  105.9512  43.275673 -76.939663    BLUFF8
7     18  4.793232e+06  342607.9388  106.1509  43.275462 -76.939658    BLUFF9
8     19  4.793233e+06  342598.2427  106.3250  43.275472 -76.939777   BLUFF10
9     20  4.793238e+06  342585.8629  106.0753  43.275516 -76.939931   BLUFF11
10    21  4.793230e+06  342578.3045  106.0633  43.275437 -76.940022   BLUFF12
11    22  4.793211e+06  342585.1251  106.7299  43.275270 -76.939

In [9]:
# TODO - add a bilinear interpolation scheme.
def bilint(z,en):
    """
    https://en.wikipedia.org/wiki/Bilinear_interpolation for a unit square
    """
    r = en[1]
    c = en[0]
    ir = np.int(r)
    ic = np.int(c)
    rx = r-ir
    cy = c-ic
    #print(r,ir,rx,'  ',c,ic,cy)
    #print(z[ir,ic],z[ir+1,ic])
    #print(z[ir+1,ic],z[ir+1,ic+1])
    zi = z[ir,ic]*(1.-cy)*(1.-rx)+z[ir+1,ic]*cy*(1.-rx)+z[ir,ic+1]*(1.-cy)*rx+z[ir+1,ic+1]*rx*cy
    return zi

In [11]:
# arrays for DEM and differences calcs
zdem = np.NaN*np.ones(len(df.index))
zdem_minus_meas = np.NaN*np.ones(len(df.index))
zdi = np.NaN*np.ones(len(df.index))
zdemi_minus_meas = np.NaN*np.ones(len(df.index))

for index, row in df.iterrows():
    # print(row["n"], row["e"])
    en = ~dem.transform*(row["e"],row["n"])
    # print( index, en )
    
    # this returns the z value of the 
    try:
        zdem[index] = z[np.int(en[1]),np.int(en[0])]
    except:
        print("out of bounds for", index, en)
    
    # this one does bilinear interpolation
    try:
        zdi[index]=bilint(z,en)
    except:
        print("interpolation problem")
    
    # No data values for DEM are -32767
    if(zdem[index]<-32766.):
        zdem[index]=np.NaN
        zdi[index]=np.NaN
    
    zdemi_minus_meas[index] = zdi[index]-row["elev"]
    zdem_minus_meas[index] = zdem[index]-row["elev"]
    #print( zdem[index], zdi[index], row["elev"], zdem_minus_meas[index]  )

out of bounds for 0 (51777.65000000037, -4852.576000005007)
interpolation problem
out of bounds for 1 (51807.89599999972, -4697.642000004649)
interpolation problem
out of bounds for 2 (51550.33999999985, -4666.733999997377)
interpolation problem
out of bounds for 3 (51327.02200000081, -4689.744000002742)
interpolation problem
out of bounds for 4 (51485.47000000067, -4352.5960000008345)
interpolation problem
out of bounds for 5 (51102.60800000094, -4255.473999992013)
interpolation problem
out of bounds for 6 (51080.66800000053, -3924.582000002265)
interpolation problem
out of bounds for 7 (51078.77600000054, -3454.3180000036955)
interpolation problem
out of bounds for 8 (50884.85400000028, -3481.7919999957085)
interpolation problem
out of bounds for 9 (50637.25800000038, -3586.0940000116825)
interpolation problem
out of bounds for 10 (50486.09000000078, -3412.662000000477)
interpolation problem
out of bounds for 11 (50622.50200000033, -3039.668000012636)
interpolation problem
out of bou

In [12]:
# add columns to the dataframe
#df.assign(DEM2=pd.Series(zdem,index=df.index))
df['DEM corner']=pd.Series(zdem,index=df.index)
df['DEM int']=pd.Series(zdi,index=df.index)
df['DEM_minus_trans']=pd.Series(zdem_minus_meas,index=df.index)
df['DEMi_minus_trans']=pd.Series(zdemi_minus_meas,index=df.index)

In [13]:
df.to_csv('Lake_Bluff_DEM_minus_trans.csv',na_rep='-32767',sep=',',float_format='% 12.4f')
df

,id,n,e,elev,lat,lon,label,DEM corner,DEM int,DEM_minus_trans,DEMi_minus_trans
0,11,4.793302e+06,342642.8825,101.9704,43.276098,-76.939247,BLUFF1,NaN,NaN,NaN,NaN
1,12,4.793294e+06,342644.3948,102.1922,43.276029,-76.939226,BLUFF2,NaN,NaN,NaN,NaN
2,13,4.793292e+06,342631.5170,104.3634,43.276012,-76.939384,BLUFF3,NaN,NaN,NaN,NaN
3,14,4.793293e+06,342620.3511,104.9136,43.276020,-76.939522,BLUFF5,NaN,NaN,NaN,NaN
4,15,4.793277e+06,342628.2735,104.6322,43.275870,-76.939420,BLUFF6,NaN,NaN,NaN,NaN
5,16,4.793272e+06,342609.1304,105.3865,43.275822,-76.939654,BLUFF7,NaN,NaN,NaN,NaN
6,17,4.793255e+06,342608.0334,105.9512,43.275673,-76.939663,BLUFF8,NaN,NaN,NaN,NaN
7,18,4.793232e+06,342607.9388,106.1509,43.275462,-76.939658,BLUFF9,NaN,NaN,NaN,NaN
8,19,4.793233e+06,342598.2427,106.3250,43.275472,-76.939777,BLUFF10,NaN,NaN,NaN,NaN
9,20,4.793238e+06,342585.8629,106.0753,43.275516,-76.939931,BLUFF11,NaN,NaN,NaN,NaN


In [ ]:
df['DEM_minus_trans'].describe()
df['DEMi_minus_trans'].describe()

In [ ]:
## Histogram of the data
x = df['DEMi_minus_trans'].as_matrix()
x = x[~np.isnan(x)]
x = x[np.abs(x)<5.]

xN = len(x)
xMAD = np.mean(np.abs(x))
xmean = x.mean()
xmin = x.min()
xmax = x.max()
xRMS = np.sqrt(np.mean(x**2))

# next line must be combined with normed=False to get fractional distribution correct
weights = np.ones_like(x)/float(len(x))

n, bins, patches = plt.hist(x, bins=100, weights=weights, normed=False, facecolor='gray', alpha=0.75)
xaxmax = 2.5
xaxmin = -2.5
yaxmin = 0.
yaxmax = .3
plt.xlabel('DEM minus Survey Point (m)')
plt.ylabel('Fraction of Observations')
plt.title(ttext)
plt.axis([xaxmin, xaxmax, yaxmin, yaxmax])
plt.grid(True)
plt.text(.75*(xaxmax-xaxmin)+xaxmin,.9*(yaxmax-yaxmin)+yaxmin,'N = {}'.format(xN), fontsize=11)
plt.text(.75*(xaxmax-xaxmin)+xaxmin,.85*(yaxmax-yaxmin)+yaxmin,'Mean = {:.3f}'.format(xmean), fontsize=11)
plt.text(.75*(xaxmax-xaxmin)+xaxmin,.8*(yaxmax-yaxmin)+yaxmin,'RMS = {:.3f}'.format(xRMS), fontsize=11)
plt.text(.75*(xaxmax-xaxmin)+xaxmin,.75*(yaxmax-yaxmin)+yaxmin,'MAD = {:.3f}'.format(xMAD), fontsize=11)
plt.text(.75*(xaxmax-xaxmin)+xaxmin,.7 *(yaxmax-yaxmin)+yaxmin,'Min = {:.3f}'.format(xmin), fontsize=11)
plt.text(.75*(xaxmax-xaxmin)+xaxmin,.65*(yaxmax-yaxmin)+yaxmin,'Max = {:.3f}'.format(xmax), fontsize=11)


plt.show()

In [ ]:
plt.plot(figsize=(12,6));plt.imshow(z[::],cmap=plt.cm.gray,vmin=75,vmax=130)